In [5]:
import sys
sys.path.append('/src')

from sfm.data_reader import DataReader
dataReader = DataReader("data/delivery_area_2l/", "", "", ".png")
# dataReader = DataReader("data/statue/", "", "", ".JPG")

Found 2 images in data/delivery_area_2l/ dataset.


In [7]:
import numpy as np
import matplotlib.pyplot as plt

scale = 1.0
convert_to_rgb = False
frameL = dataReader.readFrame("{:04d}".format(0), convert_to_rgb, scale)
frameR = dataReader.readFrame("{:04d}".format(1), convert_to_rgb, scale)

# plt.figure(figsize=(10, 10))
# plt.imshow(np.hstack((frameL, frameR)))

In [8]:
import cv2

sift = cv2.xfeatures2d.SIFT_create(nfeatures=200)

keypointsL, descriptorsL = sift.detectAndCompute(frameL, mask=None)
keypointsR, descriptorsR = sift.detectAndCompute(frameR, mask=None)

keypointsImage = cv2.drawKeypoints(frameL, keypointsL, None)
# plt.figure(figsize=(10, 10))
# plt.imshow(keypointsImage)
# plt.show()

In [9]:
from sfm.matching import generateDisparityMap

frameGrayL = cv2.cvtColor(frameL, cv2.COLOR_RGB2GRAY)
frameGrayR = cv2.cvtColor(frameR, cv2.COLOR_RGB2GRAY)
disparityMap = generateDisparityMap(frameGrayL, frameGrayR)

# plt.figure(figsize=(10, 10))
# plt.imshow(disparityMap, cmap='gray')
# plt.show()

In [10]:
fx, fy = 541.764, 541.764
cx, cy = 553.682, 232.397
baseline = 59.8896
width, height = 942, 489

Q = np.array([
    [1, 0, 0, -cx],
    [0, 1, 0, -cy],
    [0, 0, 0, fx],
    [0, 0, 1/baseline, 0]
])
print(Q)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -5.53682000e+02]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -2.32397000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.41764000e+02]
 [ 0.00000000e+00  0.00000000e+00  1.66973899e-02  0.00000000e+00]]


In [11]:
image3D = cv2.reprojectImageTo3D(disparityMap, Q, handleMissingValues=True)

pointsColors = []
for y in range(height):
    for x in range(width):
        point3D = image3D[y, x]
        if point3D[2] != 10000:
            color3D = frameL[y, x]
            pointsColors.append((point3D[0], point3D[1], point3D[2], color3D[0], color3D[1], color3D[2]))

print(pointsColors[0])

(-228.37987, -98.71038, 230.11368, 51, 51, 51)


In [12]:
from sfm.point_cloud import savePointCloud
savePointCloud(pointsColors, "point_cloud.ply")

Saved point cloud to point_cloud.ply


In [14]:
# K = np.array([[fx, 0, cx], [0, fx, cy], [0, 0, 1]])

# pointsL = np.array([x.pt for x in keypointsL])
# pointsR = np.array([x.pt for x in keypointsR])
# pointsL_norm = cv2.undistortPoints(np.expand_dims(pointsL, axis=1), cameraMatrix=K, distCoeffs=None)
# pointsR_norm = cv2.undistortPoints(np.expand_dims(pointsR, axis=1), cameraMatrix=K, distCoeffs=None)

# E, mask = cv2.findEssentialMat(pointsR_norm, pointsL_norm, focal=fx, pp=(cx,cy), method=cv2.RANSAC)
# print(E)

[[-0.08931355  0.4931712   0.12053946]
 [-0.42712554 -0.15412414 -0.50245128]
 [-0.27462999  0.43777124 -0.09253343]]


In [16]:
# _, R, T, mask = cv2.recoverPose(E, pointsR, pointsL, focal=fx, pp=(cx,cy), mask=mask)
# print(R, "\n", T)

[[ 0.98477532  0.16489675  0.05501477]
 [-0.15365781  0.97372686 -0.16806334]
 [-0.08128246  0.15705118  0.98423985]] 
 [[ 0.68450784]
 [ 0.28758385]
 [-0.66988398]]
